In [2]:
pip install pymongo pyodbc pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pymongo
import pyodbc
import pandas as pd
import re

# 1. Conectar a MongoDB local
cliente_mongo = pymongo.MongoClient("mongodb://localhost:27017/")
db_mongo = cliente_mongo['PROYECTO_ANALISIS_UNIDO'] 

# 2. Conectar a SQL Server 
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost;"  
    "DATABASE=PROYECTO_ANALISIS_UNIDO;"  
    "Trusted_Connection=yes;"
)

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# 3. Procesar todas las colecciones
for nombre_col in db_mongo.list_collection_names():
    print(f" Procesando colección: {nombre_col}")

    docs = list(db_mongo[nombre_col].find())
    for d in docs:
        d.pop('_id', None)

    if not docs:
        print(f" Colección '{nombre_col}' está vacía. Se omite.")
        continue

    # Convertir a DataFrame
    df = pd.DataFrame(docs)

    # 4. Limpiar nombres de columnas para SQL Server
    df.columns = [re.sub(r'\W+', '_', col) for col in df.columns]

    # 5. Crear tabla SQL con columnas tipo VARCHAR(MAX)
    columnas_sql = ",\n".join([f"[{col}] VARCHAR(MAX)" for col in df.columns])
    crear_tabla_sql = f"""
        IF OBJECT_ID('{nombre_col}', 'U') IS NULL
        CREATE TABLE [{nombre_col}] (
            {columnas_sql}
        )
    """
    cursor.execute(crear_tabla_sql)
    conn.commit()

    # 6. Insertar datos usando placeholders seguros
    placeholders = ", ".join(["?" for _ in df.columns])
    insert_sql = f"INSERT INTO [{nombre_col}] ({', '.join(df.columns)}) VALUES ({placeholders})"

    for _, fila in df.iterrows():
        valores = fila.fillna("").tolist()
        cursor.execute(insert_sql, *valores)

    conn.commit()
    print(f" Insertados {len(df)} registros en tabla '{nombre_col}'.")

# Cerrar conexión
cursor.close()
conn.close()
print(" Migración completa de MongoDB local a SQL Server.")


 Procesando colección: Eventos_Deportivos_2
 Insertados 316834 registros en tabla 'Eventos_Deportivos_2'.
 Procesando colección: Eventos_Noticias_Mundiales_2
 Insertados 2479 registros en tabla 'Eventos_Noticias_Mundiales_2'.
 Procesando colección: Actividades_Hobbies_3
 Insertados 9999 registros en tabla 'Actividades_Hobbies_3'.
 Procesando colección: Definido_Grupo_Vino_Reviews
 Insertados 169652 registros en tabla 'Definido_Grupo_Vino_Reviews'.
 Procesando colección: Eventos_Publicos_1
 Insertados 19702 registros en tabla 'Eventos_Publicos_1'.
 Procesando colección: Eventos_Publicos_2
 Insertados 11750 registros en tabla 'Eventos_Publicos_2'.
 Procesando colección: Actividades_Hobbies_2
 Insertados 973 registros en tabla 'Actividades_Hobbies_2'.
 Procesando colección: Actividades_Hobbies
 Insertados 32000 registros en tabla 'Actividades_Hobbies'.
 Procesando colección: Eventos_Noticias_Mundiales_1
 Insertados 499999 registros en tabla 'Eventos_Noticias_Mundiales_1'.
 Procesando cole

ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Incorrect syntax near the keyword 'key'. (156) (SQLExecDirectW); [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Statement(s) could not be prepared. (8180)")

In [4]:
import pandas as pd
from pymongo import MongoClient
import pyodbc

# 1. Conectar a MongoDB local
cliente = MongoClient("mongodb://localhost:27017/")
db_mongo = cliente["PROYECTO_ANALISIS_UNIDO"]        
coleccion = db_mongo["Eventos_Deportivos_1"]       

# 2. Leer la colección como DataFrame
datos = list(coleccion.find())
df = pd.DataFrame(datos)

# 3. Eliminar la columna '_id' 
if '_id' in df.columns:
    df = df.drop(columns=['_id'])

# 4. Conectar a SQL Server
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost;"                        
    "DATABASE=PROYECTO_ANALISIS_UNIDO;"               
    "Trusted_Connection=yes;"                  
   
)

# 5. Crear cursor
cursor = conn.cursor()

# 6. Crear tabla si no existe 
nombre_tabla_sql = "Eventos_Deportivos_1"
columnas = ", ".join([f"[{col}] NVARCHAR(MAX)" for col in df.columns])
cursor.execute(f"IF OBJECT_ID('{nombre_tabla_sql}', 'U') IS NULL CREATE TABLE {nombre_tabla_sql} ({columnas})")
conn.commit()

# 7. Insertar datos en SQL Server
for _, fila in df.iterrows():
    valores = "', '".join(str(v).replace("'", "''") for v in fila)
    query = f"INSERT INTO {nombre_tabla_sql} VALUES ('{valores}')"
    cursor.execute(query)
conn.commit()

print("Colección exportada exitosamente a SQL Server.")


Colección exportada exitosamente a SQL Server.


In [5]:
import pandas as pd
from pymongo import MongoClient
import pyodbc

# 1. Conectar a MongoDB local
cliente = MongoClient("mongodb://localhost:27017/")
db_mongo = cliente["PROYECTO_ANALISIS_UNIDO"]        
coleccion = db_mongo["Eventos_Noticias_Mundiales_3"]       

# 2. Leer la colección como DataFrame
datos = list(coleccion.find())
df = pd.DataFrame(datos)

# 3. Eliminar la columna '_id' 
if '_id' in df.columns:
    df = df.drop(columns=['_id'])

# 4. Conectar a SQL Server
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost;"                        
    "DATABASE=PROYECTO_ANALISIS_UNIDO;"               
    "Trusted_Connection=yes;"                  
   
)

# 5. Crear cursor
cursor = conn.cursor()

# 6. Crear tabla si no existe 
nombre_tabla_sql = "Eventos_Noticias_Mundiales_3"
columnas = ", ".join([f"[{col}] NVARCHAR(MAX)" for col in df.columns])
cursor.execute(f"IF OBJECT_ID('{nombre_tabla_sql}', 'U') IS NULL CREATE TABLE {nombre_tabla_sql} ({columnas})")
conn.commit()

# 7. Insertar datos en SQL Server
for _, fila in df.iterrows():
    valores = "', '".join(str(v).replace("'", "''") for v in fila)
    query = f"INSERT INTO {nombre_tabla_sql} VALUES ('{valores}')"
    cursor.execute(query)
conn.commit()

print("Colección exportada exitosamente a SQL Server.")


Colección exportada exitosamente a SQL Server.
